# Tarea 8

## Importación de Librerías

In [1]:
#------------------------------------------------------------------------
# Importar librerías
#------------------------------------------------------------------------
# manejo de datos
import pandas as pd
import numpy as np

# visualización
import matplotlib.pyplot as plt
import seaborn as sns

# preprocesamiento de datos
from sklearn.model_selection import train_test_split

# pipelines de preprocesamiento
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector, make_column_transformer, ColumnTransformer

# transformadores personalizados
from sklearn.base import BaseEstimator, TransformerMixin

# modelos
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans

# ajuste de hiperparámetros
from sklearn.model_selection import GridSearchCV # Evalúa todas las combinaciones posibles de los valores de hiperparámetros proporcionados. Determinista.
from sklearn.model_selection import RandomizedSearchCV # Búsqueda aleatoria sobre un espacio de hiperparámetros especificado. Evalúa un número fijo de combinaciones aleatorias de los valores de hiperparámetros proporcionados.

# métricas
from sklearn.metrics.pairwise import rbf_kernel # calcula el kernel rbf (gaussiano) entre X e Y
from sklearn.metrics import root_mean_squared_error # RMSE es la raíz cuadrada del promedio del cuadrado de todos los errores.

# guardar modelo
import joblib


## Custom Transformers

In [2]:
#------------------------------------------------------------------------
# Transformador personalizado para calcular la similitud con el centro del clúster
# Se utiliza el clustering de KMeans para agrupar los datos y luego se calcula la similitud de cada punto de datos con cada centro de clúster utilizando el kernel RBF.
# El kernel RBF es una función kernel popular utilizada en algoritmos de aprendizaje kernelizados.
# Parámetros de KMeans: n_clusters, n_init, random_state
# n_clusters: número de clústeres
# n_init: número de veces que se ejecutará el algoritmo KMeans con diferentes semillas de centroides
# random_state: semilla aleatoria para reproducibilidad
# Parámetro del kernel RBF: gamma
# El transformador tiene dos métodos: fit y transform
# El método fit calcula los centros de los clústeres utilizando KMeans
# El método transform calcula la similitud de cada punto de datos con cada centro de clúster utilizando el kernel RBF
# El método get_feature_names_out devuelve los nombres de las características de salida
# El transformador se puede usar en un pipeline con otros transformadores y un estimador
# El transformador se puede usar en una búsqueda en cuadrícula (grid search) para encontrar los hiperparámetros óptimos
#------------------------------------------------------------------------

class ClusterSimilarity(BaseEstimator, TransformerMixin): 
    def __init__(self, n_clusters=10, gamma=1.0, random_state=None):
        self.n_clusters = n_clusters
        self.gamma = gamma # Ancho de banda del kernel RBF
        self.random_state = random_state

    def fit(self, X, y=None, sample_weight=None):
        self.kmeans_ = KMeans(self.n_clusters, n_init=10, 
                              random_state=self.random_state)
        self.kmeans_.fit(X, sample_weight=sample_weight)
        return self  # ¡siempre devuelve self!

    def transform(self, X):
        return rbf_kernel(X, self.kmeans_.cluster_centers_, gamma=self.gamma)
    
    def get_feature_names_out(self, names=None):
        return [f"Similitud con el clúster {i}" for i in range(self.n_clusters)]

## Funciones Auxiliares

### División de los datos

In [ ]:
#------------------------------------------------------------------------
# División del conjunto de datos
# Dado que usamos GridSearch, no necesitamos incluir un conjunto de validación
#
# Eliminamos las filas sin valor en la columna objetivo
# También creamos una nueva característica "income_cat" para estratificar los datos
# y asegurarnos de que el conjunto de prueba sea representativo de las diversas categorías de ingresos en todo el conjunto de datos
# Luego eliminamos la característica "income_cat"
#------------------------------------------------------------------------

def split_data(df, test_size=0.2, random_state=42):
    """
    Cargar el conjunto de datos y dividirlo en conjuntos de entrenamiento y prueba.
    
    Args:
        df (DataFrame): Conjunto de datos
        test_size (float): Fracción de datos reservada para pruebas.
        random_state (int): Semilla para reproducibilidad.
        
    Returns:
        X_train, X_test, y_train, y_test: Conjuntos de datos divididos.
    """
    # Eliminar filas sin valor en la columna objetivo.
    # Esas filas no son utilizables para el entrenamiento (sin etiqueta) ni para la evaluación de prueba.
    df.dropna(subset=["median_house_value"], inplace=True)
    
    # Dividir en conjuntos de entrenamiento y prueba
    train_set, test_set = train_test_split(df, 
                                        test_size=0.2,
                                        stratify=pd.cut(df["median_income"],
                                                            bins=[0., 1.5, 3.0, 4.5, 6., np.inf], # Secuencia de valores que definen los límites de los intervalos
                                                            labels=[1, 2, 3, 4, 5]), # Etiquetas para cada categoría               
                                        random_state=42
                                        )

    # Eliminar la columna objetivo del conjunto de entrenamiento y preparar los conjuntos de prueba con solo la columna objetivo
    X_train = train_set.drop("median_house_value", axis=1)
    y_train = train_set["median_house_value"].copy()
    
    X_test = test_set.drop("median_house_value", axis=1)
    y_test = test_set["median_house_value"].copy()
    
    return X_train, X_test, y_train, y_test

### Ratios

In [ ]:
def column_ratio(X): # Función personalizada para calcular el ratio entre dos columnas
    return X[:, [0]] / X[:, [1]]

def ratio_name(function_transformer, feature_names_in): # Función personalizada para nombrar la nueva característica
    return ["ratio"]  # Devuelve una lista con el nombre de la nueva característica

### Pipeline de Preprocesamiento

In [ ]:
#------------------------------------------------------------------------
# Construir un pipeline de preprocesamiento para diferentes tipos de características
#
# La clase 'Pipeline' permite crear objetos que representen secuencias aplicables a cualquier conjunto de datos.
# El constructor de la clase 'Pipeline' recibe una lista de tuplas formadas por el nombre que identifica a cada estimador y ese estimador. 
# Todos los estimadores excepto el último deben ser transformadores. El último puede ser transformador, clasificador, regresor, etc.
# Cuando llamamos al método fit de la clase 'Pipeline', se llama al método fit_transform de cada estimador secuencialmente, 
# pasando la salida del método transform de un estimador al siguiente. 
#
# La clase ColumnTransformer recibe una lista de tuplas formadas por el nombre que identifica a cada transformador y ese transformador.
# Cada transformador debe ser un objeto que tenga los métodos fit y transform.
# El constructor de la clase ColumnTransformer recibe el argumento remainder que indica 
# cómo se deben tratar las columnas que no se han especificado en la lista de tuplas.
# Por defecto, remainder='drop', lo que elimina las columnas no especificadas.
# ------------------------------------------------------------------------

'''
preprocessing = ColumnTransformer([
    ("num", num_pipeline, num_attribs), # aplicamos el pipeline numérico a las columnas numéricas
    ("cat", cat_pipeline, cat_attribs)], # aplicamos el pipeline categórico a las columnas categóricas
    remainder="passthrough" # el resto de columnas se mantienen sin cambios
)

preprocessing = make_column_transformer(
    (num_pipeline, make_column_selector(dtype_include=np.number)),
    (cat_pipeline, make_column_selector(dtype_include=object)),
)
'''

def build_preprocessing_pipeline():
    """
    Construir un pipeline de preprocesamiento para diferentes tipos de características.
    
    Returns:
        preprocessor (ColumnTransformer): Un objeto ColumnTransformer que aplica las transformaciones
                                           especificadas a las columnas correspondientes.
    """
    # Pipeline para características numéricas: imputación y escalado
    numeric_pipeline = Pipeline([
        ("impute", SimpleImputer(strategy="median")), # Transformador que imputa la mediana a los valores no disponibles
        ("standardize", StandardScaler()), # Transformador que estandariza los valores
    ])
    
    
    # Pipeline para características categóricas: imputación y codificación one-hot
    categorical_pipeline = make_pipeline(
        SimpleImputer(strategy="most_frequent"), # Transformador que imputa la moda a los valores no disponibles
        OneHotEncoder(handle_unknown="ignore")) # Transformador que codifica las categorías
    
    
    # Pipeline para características logarítmicas (crear nuevas características aplicando el logaritmo a una columna)
    log_pipeline = make_pipeline(
        SimpleImputer(strategy="median"),
        FunctionTransformer(np.log, feature_names_out="one-to-one"),
        StandardScaler())
    
    
    ratio_pipeline = make_pipeline(
            SimpleImputer(strategy="median"),
            FunctionTransformer(column_ratio, feature_names_out=ratio_name),
            StandardScaler())
    
    
    # Pipeline para similitud con el centro del clúster
    cluster_simil = ClusterSimilarity(n_clusters=10, gamma=1., random_state=42)
    
    # Combinar todos los pipelines en un ColumnTransformer
    preprocessing = ColumnTransformer(transformers=[
        ("bedrooms", ratio_pipeline, ["total_bedrooms", "total_rooms"]), # ratio entre total_bedrooms y total_rooms (nueva característica)
        ("rooms_per_house", ratio_pipeline, ["total_rooms", "households"]), # ratio entre total_rooms y households (nueva característica)
        ("people_per_house", ratio_pipeline, ["population", "households"]), # ratio entre population y households (nueva característica)
        ("log", log_pipeline, ["total_bedrooms", "total_rooms", "population", "households", "median_income"]), # logaritmo de las columnas seleccionadas (para cambiar distribuciones sesgadas por distribuciones normales)
        ("geo", cluster_simil, ["latitude", "longitude"]), # similitud con los clusters
        ("cat", categorical_pipeline, make_column_selector(dtype_include=object)), # pipeline categórico
    ],
    remainder=numeric_pipeline)  # una columna restante: housing_median_age
   

    return preprocessing

### Pipeline Completo (preprocesamiento y estimador final)

In [7]:
def build_full_pipeline(model):
    """
    Construir un pipeline completo que incluya el preprocesamiento y el estimador final.
    
    Args:
        numerical_features (list): Lista de nombres de columnas numéricas.
        categorical_features (list): Lista de nombres de columnas categóricas.
        model: Un estimador de scikit-learn (por ejemplo, RandomForestRegressor).
        
    Returns:
        full_pipeline (Pipeline): Un objeto Pipeline que primero preprocesa los datos y luego aplica el modelo.
    """
  
    # Construir el pipeline completo con preprocesamiento y el estimador
    full_pipeline = Pipeline(steps=[
        ('preprocessor', build_preprocessing_pipeline()),
        ('model', model)
    ])
    
    return full_pipeline

### Gridsearch

In [8]:
#---------------------------------------------------------------------------------
# Para ajustar hiperparámetros con GridSearchCV 
#
# GridSearchCV es una clase que implementa un método de búsqueda de hiperparámetros.
# Recibe como argumentos el estimador, el espacio de parámetros, la estrategia de validación cruzada y la métrica de evaluación.
# En este caso, se realiza una búsqueda de hiperparámetros para el preprocesamiento y el modelo Random Forest Regressor.
# Se realiza una validación cruzada de 3 particiones y se utiliza la métrica de error cuadrático medio negativo.
# Se ajusta el modelo a los datos de entrenamiento.
#---------------------------------------------------------------------------------

def perform_grid_search(model, param_grid, X_train, y_train, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1):

    """
    Realizar ajuste de hiperparámetros utilizando GridSearchCV.

    Args:
        model: Un estimador de scikit-learn (por ejemplo, RandomForestRegressor).
        param_grid (dict): Diccionario con nombres de parámetros como claves y listas de configuraciones de parámetros a probar.
        X_train (DataFrame o array-like): Características de los datos de entrenamiento.
        y_train (Series o array-like): Objetivo de los datos de entrenamiento.
        cv (int): Número de particiones para validación cruzada. Por defecto es 5.
        scoring (str): Métrica de evaluación. Por defecto es 'neg_mean_squared_error'.
        verbose (int): Controla la verbosidad. Por defecto es 1.
        n_jobs (int): Número de trabajos a ejecutar en paralelo. Por defecto es -1 (usar todos los procesadores).

    Returns:
        best_model: El mejor estimador encontrado por la búsqueda en cuadrícula.
        best_params (dict): Los mejores hiperparámetros encontrados.
        best_score (float): La mejor puntuación lograda durante la validación cruzada.
    """
    
    # Instanciar GridSearchCV con el modelo y el espacio de parámetros dados
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=cv,
        scoring=scoring, 
        verbose=verbose,
        n_jobs=n_jobs
        )

    # Ajustar la búsqueda en cuadrícula a los datos de entrenamiento
    grid_search.fit(X_train, y_train)
    
    # Imprimir los mejores parámetros y la mejor puntuación
    print(f"\nMejores Parámetros: {grid_search.best_params_}")
    print(f"Mejor Puntuación: {grid_search.best_score_:.4f}") # neg_mean_squared_error 
    
    return grid_search.best_estimator_, grid_search.best_params_, grid_search.best_score_


### Random Search

In [9]:
#------------------------------------------------------------------------
# Configuración de RandomizedSearchCV
#------------------------------------------------------------------------
def perform_random_search(model, param_distributions, X_train, y_train, n_iter=100, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, random_state=42):

    """
    Realizar ajuste de hiperparámetros utilizando RandomizedSearchCV.

    Args:
        model: Un estimador de scikit-learn (por ejemplo, RandomForestRegressor).
        param_distributions (dict): Diccionario con nombres de parámetros como claves y listas o distribuciones de configuraciones de parámetros a probar.
        X_train (DataFrame o array-like): Características de los datos de entrenamiento.
        y_train (Series o array-like): Objetivo de los datos de entrenamiento.
        cv (int): Número de particiones para validación cruzada. Por defecto es 5.
        scoring (str): Métrica de evaluación. Por defecto es 'neg_mean_squared_error'.
        verbose (int): Controla la verbosidad. Por defecto es 1.
        n_iter (int): Número de configuraciones de parámetros que se prueban. Por defecto es 10.
        random_state (int): Semilla para reproducibilidad. Por defecto es 42.
        n_jobs (int): Número de trabajos a ejecutar en paralelo. Por defecto es -1 (usar todos los procesadores).

    Returns:
        best_model: El mejor estimador encontrado por la búsqueda aleatoria.
        best_params (dict): Los mejores hiperparámetros encontrados.
        best_score (float): La mejor puntuación lograda durante la validación cruzada.
    """
    
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=n_iter,  # Número de combinaciones aleatorias a probar
        cv=cv,  # Validación cruzada
        scoring=scoring,
        n_jobs=n_jobs,  # Paralelización de cálculos
        random_state=random_state  # Semilla para reproducibilidad
    )

    # Entrenar el modelo
    random_search.fit(X_train, y_train)
    
    # Imprimir los mejores parámetros y la mejor puntuación
    print(f"\nMejores Parámetros: {random_search.best_params_}")
    print(f"Mejor Puntuación: {random_search.best_score_:.4f}")

    return random_search.best_estimator_, random_search.best_params_, random_search.best_score_

### Evaluar Modelo

In [10]:
#------------------------------------------------------------------------
# Para evaluar un modelo
# También imprime métricas de rendimiento
#------------------------------------------------------------------------
def evaluate_model(model, X_test, y_test):
    """
    Evaluar el modelo e imprimir métricas de rendimiento.
    
    Args:
        model: Modelo de aprendizaje automático entrenado.
        X_test (DataFrame): Características del conjunto de prueba.
        y_test (Series o array-like): Valores objetivo verdaderos.
    """
    # Realizar predicciones
    y_pred = model.predict(X_test)
    
    # Evaluación de regresión:
    print(f"Error cuadrático medio (RMSE) en el CONJUNTO DE PRUEBA: {root_mean_squared_error(y_test, y_pred):.4f}")
    
    # Si esto fuera un problema de clasificación, podrías hacer:
    # print("Precisión:", accuracy_score(y_test, y_pred))
    # print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))
    # print("Informe de clasificación:\n", classification_report(y_test, y_pred))


## Carga y preprocesado

In [11]:
#------------------------------------------------------------------------
# Carga del conjunto de datos + verifica información
#------------------------------------------------------------------------

housing = pd.read_csv("./data/housing.csv")

housing.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [12]:
housing.head() # verifica las primeras 5 filas del conjunto de datos

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [13]:
#------------------------------------------------------------------------
# División del conjunto de datos
#------------------------------------------------------------------------

X_train, X_test, y_train, y_test = split_data(housing, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test

(       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
 12655    -121.46     38.52                29.0       3873.0           797.0   
 15502    -117.23     33.09                 7.0       5320.0           855.0   
 2908     -119.04     35.37                44.0       1618.0           310.0   
 14053    -117.13     32.75                24.0       1877.0           519.0   
 20496    -118.70     34.28                27.0       3536.0           646.0   
 ...          ...       ...                 ...          ...             ...   
 15174    -117.07     33.03                14.0       6665.0          1231.0   
 12661    -121.42     38.51                15.0       7901.0          1422.0   
 19263    -122.72     38.44                48.0        707.0           166.0   
 19140    -122.70     38.31                14.0       3155.0           580.0   
 19773    -122.14     39.97                27.0       1079.0           222.0   
 
        population  households  median

In [27]:
#------------------------------------------------------------------------
# Posibles hiperparámetros configurables de Random Forest Regressor
#
#  n_estimator              El número de árboles en el bosque
#  criterion                Decide cómo se divide el árbol: 'gini', 'entropy'. Función para medir la calidad de una división
#  max_depth                La profundidad máxima del árbol
#  min_samples_split        El número mínimo de muestras requerido para dividir un nodo interno
#  min_samples_leaf         El número mínimo de muestras requerido para estar en un nodo hoja
#  min_weight_fraction_leaf La fracción mínima ponderada del total de pesos (de todas las muestras de entrada) requerida para estar en un nodo hoja
#  max_features             El número de características a considerar al buscar la mejor división
#  max_leaf_nodes           Crece árboles con max_leaf_nodes en el modo de mejor-primer
#  min_impurity_decrease    Un nodo se dividirá si esta división induce una disminución de la impureza mayor o igual a este valor
#  bootstrap                Indica si se utilizan muestras bootstrap al construir los árboles
#  oob_score                Indica si se deben usar muestras fuera de bolsa (out-of-bag) para estimar el error de generalización
#  n_jobs                   El número de trabajos a ejecutar en paralelo
#  random_state             Controla tanto la aleatoriedad del muestreo bootstrap de las muestras utilizadas al construir los árboles
#                           como el muestreo de las características a considerar al buscar la mejor división en cada nodo
#  verbose                  Controla la verbosidad al ajustar y predecir
#  warm_start               Cuando se establece en True, reutiliza la solución de la llamada anterior a fit y agrega más estimadores al conjunto
#  ccp_alpha                Parámetro de complejidad utilizado para la poda de costo-minimalidad
#  max_samples              Si bootstrap es True, el número de muestras a extraer de X para entrenar cada estimador base
#  monotonic_cst            Indica la restricción de monotonía a aplicar en cada característica.
#------------------------------------------------------------------------

full_pipeline = build_full_pipeline(RandomForestRegressor(random_state=42))

full_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder=Pipeline(steps=[('impute',
                                                              SimpleImputer(strategy='median')),
                                                             ('standardize',
                                                              StandardScaler())]),
                                   transformers=[('bedrooms',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(feature_names_out=<function ratio_name at 0x0000017E172623E0>,...
                                                   'median_income']),
                                                 ('geo',
                                                  ClusterSimilarity(random_state=42),
                                                  ['latitude', 'longitude']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000017E17330170>)])),
                ('model', RandomForestRegressor(random_state=42))])

In [15]:
# Cada parámetro en el param_grid debe coincidir exactamente con una de estas claves
print(full_pipeline.get_params().keys())


dict_keys(['memory', 'steps', 'transform_input', 'verbose', 'preprocessor', 'model', 'preprocessor__force_int_remainder_cols', 'preprocessor__n_jobs', 'preprocessor__remainder__memory', 'preprocessor__remainder__steps', 'preprocessor__remainder__transform_input', 'preprocessor__remainder__verbose', 'preprocessor__remainder__impute', 'preprocessor__remainder__standardize', 'preprocessor__remainder__impute__add_indicator', 'preprocessor__remainder__impute__copy', 'preprocessor__remainder__impute__fill_value', 'preprocessor__remainder__impute__keep_empty_features', 'preprocessor__remainder__impute__missing_values', 'preprocessor__remainder__impute__strategy', 'preprocessor__remainder__standardize__copy', 'preprocessor__remainder__standardize__with_mean', 'preprocessor__remainder__standardize__with_std', 'preprocessor__remainder', 'preprocessor__sparse_threshold', 'preprocessor__transformer_weights', 'preprocessor__transformers', 'preprocessor__verbose', 'preprocessor__verbose_feature_name

## Entrenar modelo con GRID SEARCH



In [ ]:
#------------------------------------------------------------------------
# Búsqueda en cuadrícula (Grid Search)
#------------------------------------------------------------------------

# Más reducido para ejecutar en máquinas con menos potencia
param_grid = [
    {'preprocessor__geo__n_clusters': [5, 10],
    "model__n_estimators": [100, 200],
    "model__max_depth": [None, 3],
    "model__min_samples_leaf": [1, 2],
    "model__min_samples_split": [2, 10],
    }
    ]

# Más amplio para ejecutar en máquinas con más potencia
param_grid = [
    {'preprocessor__geo__n_clusters': [5, 8, 10],
    'model__max_features': [4, 6, 8],
    'model__max_depth': [None, 3],
    'model__min_samples_leaf': [1, 2, 4],
    'model__min_samples_split': [2, 5, 10]
    }
    ]

best_model, best_params, best_score = perform_grid_search(full_pipeline, param_grid, X_train, y_train, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)


Fitting 3 folds for each of 162 candidates, totalling 486 fits

Mejores Parámetros: {'model__max_depth': None, 'model__max_features': 4, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'preprocessor__geo__n_clusters': 10}
Mejor Puntuación: -1979261647.5525


## Entrenar modelo con RANDOM SEARCH

In [ ]:
#------------------------------------------------------------------------
# Búsqueda Aleatoria
#------------------------------------------------------------------------

# Más reducido para ejecutar en máquinas con menos potencia
param_distributions = [
    {'preprocessor__geo__n_clusters': [5, 10],
    "model__n_estimators": [100, 200],
    "model__max_depth": [None, 3],
    "model__min_samples_leaf": [1, 2],
    "model__min_samples_split": [2, 10],
    }
    ]

# Más amplio para ejecutar en máquinas con más potencia
param_distributions = {'preprocessor__geo__n_clusters': [5, 8, 10],
    'model__max_features': [4, 6, 8],
    'model__max_depth': [None, 3],
    'model__min_samples_leaf': [1, 2, 4],
    'model__min_samples_split': [2, 5, 10]
    }

# establecer n_iter al mismo valor de combinaciones que GridSearchCV probará
best_model, best_params, best_score = perform_random_search(full_pipeline, param_distributions, X_train, y_train, n_iter=96, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, random_state=42)


Mejores Parámetros: {'preprocessor__geo__n_clusters': 10, 'model__min_samples_split': 2, 'model__min_samples_leaf': 1, 'model__max_features': 4, 'model__max_depth': None}
Mejor Puntuación: -1979261647.5525


In [18]:
# Para predecir y evaluar el mejor modelo
evaluate_model(best_model, X_test, y_test)

Error cuadrático medio (RMSE) en el CONJUNTO DE PRUEBA: 41742.9799


## Guardar el modelo

In [ ]:
#Ocupa mucho así que lo comento
#joblib.dump(best_model, "best_model.pkl")